In [1]:
from model import Model
import json
import os
#500m 1650000
#250m 835000
#100m 335000

/home/j1choi/Downloads/pyssem-main/pyssem/utils/simulation/scen_properties.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
# Load simulation configuration
with open('./example-purdue.json') as f:
  simulation_data = json.load(f)

In [3]:
scenario_props = simulation_data['scenario_properties']

In [4]:
scenario_props

{'start_date': '01/03/2024',
 'simulation_duration': 100,
 'steps': 100000,
 'min_altitude': 515,
 'max_altitude': 900,
 'v_shells': 680000,
 'launch_function': 'Lambda',
 'integrator': 'BDF',
 'density_model': 'static_exp_dens_func',
 'LC': 0.1,
 'v_imp': 10.0,
 'fragment_spreading': False,
 'parallel_processing': False,
 'basline': False}

In [5]:
# Create an instance of the Model with the simulation parameters
model = Model(
    start_date=scenario_props["start_date"].split("T")[0],  # Assuming date is in ISO format
    simulation_duration=scenario_props["simulation_duration"],
    steps=scenario_props["steps"],
    min_altitude=scenario_props["min_altitude"],
    max_altitude=scenario_props["max_altitude"],
    v_shells=scenario_props["v_shells"],
    launch_function=scenario_props["launch_function"],
    integrator=scenario_props["integrator"],
    density_model=scenario_props["density_model"],
    LC=scenario_props["LC"],
    v_imp=scenario_props["v_imp"],
    fragment_spreading = scenario_props["fragment_spreading"]
)

SHELL COUNT:  10
R0_km [515.         515.20394479 515.40772825 515.61135068 515.81481242
 516.01811377 516.22125506 516.42423659 516.62705868 516.82972165
 517.0322258 ]


In [6]:
species = simulation_data["species"]
species_list = model.configure_species(species)

Splitting species N into 2 species with masses [0.64, 223].
Added 1 active species, 2 debris species, and 0 rocket body species to the simulation.
Pairing the following active species to debris classes for PMD modeling...
['S']
Matched species S to debris species N_223kg.
    Name: N_0.64kg
    pmd_linked_species: []
    Name: N_223kg
    pmd_linked_species: ['S']


Creating collision pairs:  17%|███▏               | 1/6 [00:00<00:00,  8.59it/s]

Species pairing for  S N_0.64kg and sk  ['S', 'N_0.64kg', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'N_0.64kg'}
Species pairing for  S N_223kg and sk  ['S', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'N_223kg'}
Species pairing for  N_0.64kg N_223kg and sk  ['N_0.64kg', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'N_0.64kg'} {'N_223kg'}


Creating collision pairs:  83%|███████████████▊   | 5/6 [00:00<00:00, 13.66it/s]

Species pairing for  S S and sk  ['S', 'S', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'S'}
Species pairing for  N_0.64kg N_0.64kg and sk  ['N_0.64kg', 'N_0.64kg', 'N_0.64kg', 'N_223kg']
pairing {'N_0.64kg'} {'N_0.64kg'}
Species pairing for  N_223kg N_223kg and sk  ['N_223kg', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'N_223kg'} {'N_223kg'}


Creating collision pairs: 100%|███████████████████| 6/6 [00:00<00:00, 13.53it/s]


In [7]:
# Run the model
# ODE solution
results = model.run_model()

baseline
PER YEAR 31557600.0 1000.0
PER YEAR 31557600.0 1000.0
PER YEAR 31557600.0 1000.0
Conversion of equations to lambda functions...
x0           S  N_0.64kg   N_223kg
0  1.871603  0.373474   0.18716
1   1.87418  0.373559  0.187418
2  1.876754  0.373644  0.187675
3  1.879325  0.373728  0.187933
4  1.881894  0.373811  0.188189
5  1.884461  0.373894  0.188446
6  1.887025  0.373977  0.188703
7  1.889587  0.374058  0.188959
8  1.892146   0.37414  0.189215
9  1.894703   0.37422   0.18947
Integrating equations...
Model run completed successfully.


In [8]:
#model.create_plots()

In [9]:
# creating list for all species
all_species = species_list.species['active'] + species_list.species['debris']

initial_state = {}

# initial population by shell and species
for shell_idx in range(model.scenario_properties.n_shells):
    for species_idx, species_obj in enumerate(all_species):
        count = model.scenario_properties.x0.values[shell_idx, species_idx]
        initial_state[f'shell_{shell_idx+1}_{species_obj.sym_name}'] = count
        print(f'shell_{shell_idx+1}_{species_obj.sym_name}: {count}')

shell_1_S: 1.8716032436752983
shell_1_N_0.64kg: 0.37347428444158326
shell_1_N_223kg: 0.18716032436752983
shell_2_S: 1.8741795692804764
shell_2_N_0.64kg: 0.3735593538941575
shell_2_N_223kg: 0.18741795692804764
shell_3_S: 1.8767535005912457
shell_3_N_0.64kg: 0.3736438786769875
shell_3_N_223kg: 0.18767535005912456
shell_4_S: 1.8793250313058123
shell_4_N_0.64kg: 0.37372785972225875
shell_4_N_223kg: 0.18793250313058124
shell_5_S: 1.8818941551466728
shell_5_N_0.64kg: 0.3738112979612623
shell_5_N_223kg: 0.18818941551466728
shell_6_S: 1.8844608658605806
shell_6_N_0.64kg: 0.3738941943243939
shell_6_N_223kg: 0.18844608658605808
shell_7_S: 1.8870251572185133
shell_7_N_0.64kg: 0.37397654974115285
shell_7_N_223kg: 0.18870251572185132
shell_8_S: 1.8895870230156409
shell_8_N_0.64kg: 0.37405836514014107
shell_8_N_223kg: 0.18895870230156409
shell_9_S: 1.8921464570712931
shell_9_N_0.64kg: 0.3741396414490619
shell_9_N_223kg: 0.18921464570712931
shell_10_S: 1.8947034532289233
shell_10_N_0.64kg: 0.37422037

In [10]:
# define collision events
from utils.simulation.discrete_event import define_collision_events_from_pairs, define_launch_events, define_pmd_events, define_drag_events
collision_events = define_collision_events_from_pairs(model, initial_state.copy())

# define launch events
launch_events = define_launch_events(all_species, model)

# define PMD events
pmd_events = define_pmd_events(all_species, model, initial_state.copy())

# define drag events
drag_events = define_drag_events(all_species, model, initial_state.copy())

colls
{'name': 'collision_S_N_0.64kg_lethal_shell_1', 'rate': 7.93504456906753e-7, 'jump': {'shell_1_S': -1, 'shell_1_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_1', 'rate': 3.96752228453377e-5, 'jump': {'shell_1_S': -1, 'shell_1_N_223kg': 1}}
{'name': 'collision_S_N_0.64kg_lethal_shell_2', 'rate': 7.95359919183608e-7, 'jump': {'shell_2_S': -1, 'shell_2_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_2', 'rate': 3.97679959591804e-5, 'jump': {'shell_2_S': -1, 'shell_2_N_223kg': 1}}
{'name': 'collision_S_N_0.64kg_lethal_shell_3', 'rate': 7.97215283736588e-7, 'jump': {'shell_3_S': -1, 'shell_3_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_3', 'rate': 3.98607641868294e-5, 'jump': {'shell_3_S': -1, 'shell_3_N_223kg': 1}}
{'name': 'collision_S_N_0.64kg_lethal_shell_4', 'rate': 7.99070539826442e-7, 'jump': {'shell_4_S': -1, 'shell_4_N_0.64kg': 115.0}}
{'name': 'collision_S_N_0.64kg_disabling_shell_4', 'rate': 3.99535269913221e-5, 'ju

In [11]:
import numpy as np
import pandas as pd
from utils.simulation.discrete_event import update_collision_event_rates, update_drag_events, update_pmd_events


def run_des(model, initial_state, all_species, launch_events):
    state = initial_state.copy()
    time = 0
    simulation_duration = model.scenario_properties.simulation_duration
    event_log = []

    while time < simulation_duration:
        collision_events = update_collision_event_rates(
            model.scenario_properties.collision_pairs,
            state,
            model.scenario_properties.n_shells
        )
        
        drag_events = update_drag_events(all_species, model, state)
        pmd_events = update_pmd_events(all_species, model, state)

        all_events = collision_events + launch_events + pmd_events #+ drag_events

        total_rate = sum(event['rate'] for event in all_events)

        if total_rate == 0:
            break

        next_event_time = np.random.exponential(1 / total_rate)
        time += next_event_time

        probabilities = [event['rate'] / total_rate for event in all_events]
        selected_event = np.random.choice(all_events, p=probabilities)

        for species_shell, jump in selected_event['jump'].items():
            state[species_shell] += jump
            state[species_shell] = max(state[species_shell], 0)

        event_log.append({
            'time': time,
            'event': selected_event['name'],
            'state': state.copy()
        })

    scen_times = model.scenario_properties.scen_times
    df_event_log = pd.DataFrame(event_log)

    n_shells = model.scenario_properties.n_shells
    n_species = len(all_species)
    result_array = np.zeros((n_species * n_shells, len(scen_times)))

    species_shell_names = [
        f'shell_{k+1}_{sp.sym_name}' 
        for sp in all_species for k in range(n_shells)
    ]

    current_idx = 0
    for step_idx, t in enumerate(scen_times):
        while current_idx + 1 < len(df_event_log) and df_event_log['time'].iloc[current_idx + 1] <= t:
            current_idx += 1
        
        current_state = df_event_log['state'].iloc[current_idx]
        for idx, species_shell in enumerate(species_shell_names):
            result_array[idx, step_idx] = current_state.get(species_shell, 0)

    return result_array



In [ ]:
from datetime import datetime
import numpy as np

num_runs = 20
results_list = []
log_file = "iteration_log.txt"

for i in range(num_runs):
    current_time = datetime.now()
    log_line = f"Iteration {i} start time: {current_time.strftime('%Y-%m-%d %H:%M:%S')}\n"
    print(log_line)
    
    # 로그 파일에 이어쓰기
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(log_line)
    
    des_results = run_des(model, initial_state, all_species, launch_events)
    results_list.append(des_results)

# Calculate Average
mean_result_array = np.mean(results_list, axis=0)

print("Mean result array shape:", mean_result_array.shape)


Iteration 0 start time: 2025-09-28 07:47:05

Iteration 1 start time: 2025-09-28 07:48:00

Iteration 2 start time: 2025-09-28 07:48:55

Iteration 3 start time: 2025-09-28 07:49:53

Iteration 4 start time: 2025-09-28 07:50:50

Iteration 5 start time: 2025-09-28 07:51:46

Iteration 6 start time: 2025-09-28 07:52:42

Iteration 7 start time: 2025-09-28 07:53:39

Iteration 8 start time: 2025-09-28 07:54:35

Iteration 9 start time: 2025-09-28 07:55:31

Iteration 10 start time: 2025-09-28 07:56:28

Iteration 11 start time: 2025-09-28 07:57:26

Iteration 12 start time: 2025-09-28 07:58:22

Iteration 13 start time: 2025-09-28 07:59:19

Iteration 14 start time: 2025-09-28 08:00:17

Iteration 15 start time: 2025-09-28 08:01:12

Iteration 16 start time: 2025-09-28 08:02:10

Iteration 17 start time: 2025-09-28 08:03:05



In [ ]:
results.output.y

In [ ]:
mean_result_array

In [ ]:
results.output.y = mean_result_array

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

scenario_properties = model.scenario_properties
species_names = scenario_properties.species_names
num_shells = scenario_properties.n_shells
n_species = len(species_names)

# Calculate average and std from results list
results_array = np.array(results_list)  # shape: (num_runs, n_species*num_shells, n_steps)
mean_results = np.mean(results_array, axis=0)
std_results = np.std(results_array, axis=0)

# Calculate conficence interval
ci_factor = 1.96 / np.sqrt(len(results_list))  # 95% CI for the mean
ci_results = ci_factor * std_results

output_t = scenario_properties.scen_times

os.makedirs('figures', exist_ok=True)

# Plotting species population per shell (with CI)
fig, axes = plt.subplots(1, n_species, figsize=(20, 6))

cmap = plt.cm.viridis
norm = plt.Normalize(vmin=0, vmax=num_shells - 1)
colors = [cmap(norm(sh)) for sh in range(num_shells)]

for species_index in range(n_species):
    ax = axes.flatten()[species_index]
    species_data_mean = mean_results[species_index * num_shells:(species_index + 1) * num_shells]
    species_data_ci = ci_results[species_index * num_shells:(species_index + 1) * num_shells]

    for shell_index in range(num_shells):
        mean_curve = species_data_mean[shell_index]
        ci_curve = species_data_ci[shell_index]
        color = colors[shell_index]

        ax.plot(output_t, mean_curve, color=color, label=f'Shell {shell_index + 1}')
        ax.fill_between(output_t, mean_curve - ci_curve, mean_curve + ci_curve, color=color, alpha=0.2)

    ax.set_title(f'{species_names[species_index]}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

plt.suptitle('Species across All Shells with 95% CI')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('figures/species_all_shells_DES.png')
plt.close(fig)

# Plotting Total Objects (with CI)
plt.figure(figsize=(10, 6))

total_objects_all_species_mean = np.zeros_like(output_t)
total_objects_all_species_ci = np.zeros_like(output_t)

for i in range(n_species):
    start_idx = i * num_shells
    end_idx = start_idx + num_shells

    total_mean = np.sum(mean_results[start_idx:end_idx, :], axis=0)
    total_ci = np.sqrt(np.sum(ci_results[start_idx:end_idx, :] ** 2, axis=0))  # CI propagation assuming independence

    plt.plot(output_t, total_mean, label=f'{species_names[i]}')
    plt.fill_between(output_t, total_mean - total_ci, total_mean + total_ci, alpha=0.2)

    total_objects_all_species_mean += total_mean
    total_objects_all_species_ci += total_ci ** 2  # Variances add up

total_objects_all_species_ci = np.sqrt(total_objects_all_species_ci)

plt.plot(output_t, total_objects_all_species_mean, label='Total All Species', color='k', linewidth=2, linestyle='--')
plt.fill_between(output_t,
                 total_objects_all_species_mean - total_objects_all_species_ci,
                 total_objects_all_species_mean + total_objects_all_species_ci,
                 color='gray', alpha=0.2)

plt.xlabel('Time')
plt.ylabel('Total Number of Objects')
plt.title('Objects Over Time with 95% Confidence Interval')
plt.xlim(0, 100)
#plt.ylim(0, 2500)
plt.legend()
plt.tight_layout()
plt.savefig('figures/total_objects_over_time_DES.png')
plt.close()


In [ ]:
print("complete")